In [2]:
import os
import warnings
import tensorflow as tf

# Supress warnings agar output lebih bersih
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# --- KONFIGURASI GPU (WAJIB UNTUK LOKAL) ---
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Mengatur memory growth agar tidak langsung memakan 100% VRAM
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU Terdeteksi & Siap: {gpus[0].name}")
    except RuntimeError as e:
        print(e)
else:
    print("⚠️ GPU Tidak terdeteksi. Menggunakan CPU.")

⚠️ GPU Tidak terdeteksi. Menggunakan CPU.


In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import layers, models, Model
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# --- KONFIGURASI HYPERPARAMETER (OPTIMAL) ---
# Berdasarkan Paper Referensi 
LEARNING_RATE = 0.000506
NEURONS_FLN = 471
DROPOUT_RATE = 0.44
BATCH_SIZE = 16   # Sesuaikan dengan VRAM (8, 16, atau 32)
EPOCHS = 30       # Epoch maksimal (akan berhenti jika early stopping aktif)
IMG_SIZE = (224, 224)
NUM_CLASSES = 3   # Benign, Malignant, Normal
DATA_DIR = "D:\\Kuliah\\Semester 5\\HUMIC\\dataset gabungan" # Pastikan folder dataset ada di sini

In [4]:
def preprocessing_clahe_densenet(img):
    """
    Pipeline Preprocessing:
    1. Convert ke LAB Color Space
    2. Terapkan CLAHE pada channel L (Lightness)
    3. Convert balik ke RGB
    4. Normalisasi standar DenseNet (tf.keras.applications.densenet.preprocess_input)
    """
    # Pastikan format uint8
    img = img.astype('uint8')
    
    # 1. RGB -> LAB
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    
    # 2. Terapkan CLAHE (ClipLimit 2.0, Grid 8x8 sesuai paper) [cite: 98]
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    
    # 3. Gabungkan & Balik ke RGB
    limg = cv2.merge((cl, a, b))
    final_img = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
    
    # 4. Normalisasi DenseNet (Scaling input ke range spesifik ImageNet)
    final_img = tf.keras.applications.densenet.preprocess_input(final_img)
    
    return final_img

In [5]:
# Setup Generator dengan Augmentasi
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocessing_clahe_densenet, # Custom Function di atas
    rotation_range=20,        # Rotasi [cite: 108]
    width_shift_range=0.1,    # Geser Horizontal [cite: 109]
    height_shift_range=0.1,   # Geser Vertikal
    horizontal_flip=True,     # Valid secara medis [cite: 110]
    vertical_flip=False,      # JANGAN vertical flip (Bayangan akustik) [cite: 112]
    zoom_range=0.2,           # Simulasi ukuran tumor [cite: 114]
    validation_split=0.2      # Split 20% untuk Validasi
)

print("Menyiapkan Training Generator...")
train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

print("\nMenyiapkan Validation Generator...")
val_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False # Jangan shuffle agar evaluasi urut
)

# Cek Mapping Kelas
class_names = list(train_generator.class_indices.keys())
print(f"\nLabel Kelas: {class_names}")

Menyiapkan Training Generator...
Found 826 images belonging to 3 classes.

Menyiapkan Validation Generator...
Found 204 images belonging to 3 classes.

Label Kelas: ['test', 'train', 'validation']


In [6]:
def build_densenet_fln_model():
    # 1. Input Layer
    inputs = layers.Input(shape=(224, 224, 3))
    
    # 2. Backbone: DenseNet201
    # include_top=False membuang layer klasifikasi asli
    # pooling='avg' meratakan output menjadi vektor 1D (1920 fitur)
    base_model = DenseNet201(
        include_top=False,
        weights='imagenet',
        input_tensor=inputs,
        pooling='avg' 
    )
    
    # Bekukan bobot backbone (Transfer Learning) [cite: 135]
    base_model.trainable = False
    
    # Ambil fitur hasil ekstraksi DenseNet
    features = base_model.output
    
    # 3. Fast Learning Network (FLN) Head / DPFNN
    # --- Jalur 1: Transformasi Non-Linear ---
    hidden_path = layers.Dense(NEURONS_FLN, activation='relu', name='fln_hidden')(features)
    hidden_path = layers.Dropout(DROPOUT_RATE, name='fln_dropout')(hidden_path)
    
    # --- Jalur 2: Jalur Langsung (Direct Path) ---
    # Fitur asli (features) langsung digabungkan dengan output jalur 1
    # Ini kunci dari "Fast Learning" untuk mempercepat konvergensi [cite: 152]
    concatenated = layers.concatenate([hidden_path, features], name='fln_concat')
    
    # 4. Output Layer
    outputs = layers.Dense(NUM_CLASSES, activation='softmax', name='prediction')(concatenated)
    
    model = Model(inputs=inputs, outputs=outputs, name="DenseNet201_FLN_Hybrid")
    return model

# Build & Summary
model = build_densenet_fln_model()
model.summary()

Model: "DenseNet201_FLN_Hybrid"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,408 │ zero_padding2d[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_1    │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 56, 56,    │          0 │ zero_padding2d_1… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 56, 56,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 56, 56,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 56, 56,    │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, 56, 56,    │          0 │ conv2_block2_0_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, 56, 56,    │     12,288 │ conv2_block2_0_r

 Total params: 19,233,951 (73.37 MB)

 Trainable params: 911,967 (3.48 MB)

 Non-trainable params: 18,321,984 (69.89 MB)

In [7]:
# 1. Model Checkpoint: Simpan model hanya jika akurasi validasi naik
checkpoint = ModelCheckpoint(
    'best_model_densenet_fln.keras', 
    monitor='val_accuracy', 
    save_best_only=True, 
    mode='max', 
    verbose=1
)

# 2. Early Stopping: Berhenti training jika val_accuracy stuck selama 8 epoch
early_stopping = EarlyStopping(
    monitor='val_accuracy', 
    patience=8, 
    restore_best_weights=True,
    verbose=1
)

# 3. ReduceLROnPlateau: Turunkan LR jika stuck, agar bisa fine-tuning [Request User]
lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,       # Kurangi LR jadi setengahnya
    patience=3,       # Tunggu 3 epoch sebelum menurunkan
    min_lr=1e-6,      # Batas bawah LR
    verbose=1
)

callbacks_list = [checkpoint, early_stopping, lr_scheduler]

In [ ]:
# Compile Model
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')]
)

print("🚀 Memulai Training...")
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=callbacks_list
)
print("✅ Training Selesai.")

🚀 Memulai Training...
Epoch 1/30
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 464ms/step - accuracy: 0.7442 - loss: 0.7762 - precision: 0.7539 - recall: 0.7248
Epoch 1: val_accuracy improved from None to 0.85417, saving model to best_model_densenet_fln.keras
51/51 ━━━━━━━━━━━━━━━━━━━━ 63s 973ms/step - accuracy: 0.7757 - loss: 0.7121 - precision: 0.7838 - recall: 0.7598 - val_accuracy: 0.8542 - val_loss: 0.5019 - val_precision: 0.8542 - val_recall: 0.8542 - learning_rate: 5.0600e-04
Epoch 2/30
 1/51 ━━━━━━━━━━━━━━━━━━━━ 19s 394ms/step - accuracy: 0.5000 - loss: 1.7937 - precision: 0.5556 - recall: 0.5000
Epoch 2: val_accuracy did not improve from 0.85417
51/51 ━━━━━━━━━━━━━━━━━━━━ 13s 257ms/step - accuracy: 0.5000 - loss: 1.7937 - precision: 0.5556 - recall: 0.5000 - val_accuracy: 0.8542 - val_loss: 0.5015 - val_precision: 0.8542 - val_recall: 0.8542 - learning_rate: 5.0600e-04
Epoch 3/30
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7811 - loss: 0.6514 - precision: 0.7943 - recall: 0.7691
Epoch

In [2]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    # Ambil history Learning Rate
    lr = history.history.get('lr', history.history.get('learning_rate'))
    
    epochs_range = range(len(acc))
    
    plt.figure(figsize=(18, 5))
    
    # Plot Akurasi
    plt.subplot(1, 3, 1)
    plt.plot(epochs_range, acc, label='Train Accuracy')
    plt.plot(epochs_range, val_acc, label='Val Accuracy')
    plt.title('Akurasi Model')
    plt.legend()
    plt.grid(True)
    
    # Plot Loss
    plt.subplot(1, 3, 2)
    plt.plot(epochs_range, loss, label='Train Loss')
    plt.plot(epochs_range, val_loss, label='Val Loss')
    plt.title('Loss Model')
    plt.legend()
    plt.grid(True)
    
    # Plot Learning Rate
    plt.subplot(1, 3, 3)
    plt.plot(epochs_range, lr, label='Learning Rate', color='orange', marker='o')
    plt.title('Dinamika Learning Rate')
    plt.yscale('log') # Skala logaritmik
    plt.xlabel('Epoch')
    plt.ylabel('LR')
    plt.grid(True, which="both", ls="-", alpha=0.5)
    
    plt.tight_layout()
    plt.show()

plot_history(history)

NameError: name 'history' is not defined

In [ ]:
# Load model terbaik (bukan model akhir training)
best_model = tf.keras.models.load_model('best_model_densenet_fln.keras')

print("Melakukan Prediksi pada Data Validasi...")
# Prediksi
predictions = best_model.predict(val_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = val_generator.classes

# Classification Report
print("\n📄 Laporan Klasifikasi:")
print(classification_report(y_true, y_pred, target_names=class_names))

# Confusion Matrix Heatmap
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Prediksi')
plt.ylabel('Aktual')
plt.title('Confusion Matrix')
plt.show()